In [546]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# import the necessary packages

In [547]:
target_column = 'Price'

In [548]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 150)

In [549]:
def handleOriginal(original, test_df):
    original.insert(0, 'id', range(test_df['id'].max()+1, test_df['id'].max() + 1 + len(original)))

In [550]:
train = pd.read_csv("train.csv")
train_extra = pd.read_csv("training_extra.csv")
original = pd.read_csv("original.csv")
test = pd.read_csv("test.csv")

handleOriginal(original, train_extra)

train['Dataset'] = 'train'
train_extra['Dataset'] = 'train_extra'
original['Dataset'] = 'original'
test['Dataset'] = 'test'

data = pd.concat([train, test, train_extra, original]).reset_index(drop=True)

In [551]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4246818 entries, 0 to 4246817
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Brand                 object 
 2   Material              object 
 3   Size                  object 
 4   Compartments          float64
 5   Laptop Compartment    object 
 6   Waterproof            object 
 7   Style                 object 
 8   Color                 object 
 9   Weight Capacity (kg)  float64
 10  Price                 float64
 11  Dataset               object 
dtypes: float64(3), int64(1), object(8)
memory usage: 388.8+ MB


In [552]:
def shift_target_column_to_end(data):
    target_data = data.pop(target_column)
    data[target_column] = target_data
    return data

In [553]:
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].astype('category')

In [554]:
for col in data.columns:
    if data[col].dtype != 'category' or col == 'Dataset':
        continue
    data[col] = data[col].cat.codes
    data[col] = data[col].astype('category')

In [555]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4246818 entries, 0 to 4246817
Data columns (total 12 columns):
 #   Column                Dtype   
---  ------                -----   
 0   id                    int64   
 1   Brand                 category
 2   Material              category
 3   Size                  category
 4   Compartments          float64 
 5   Laptop Compartment    category
 6   Waterproof            category
 7   Style                 category
 8   Color                 category
 9   Weight Capacity (kg)  float64 
 10  Price                 float64 
 11  Dataset               category
dtypes: category(8), float64(3), int64(1)
memory usage: 162.0 MB


In [556]:
data.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,Dataset
0,0,1,1,1,7.0,1,0,2,0,11.611723,112.15875,train
1,1,1,0,2,10.0,1,1,1,3,27.078537,68.88056,train
2,2,4,1,2,2.0,1,0,1,5,16.643760,39.17320,train
3,3,2,2,2,8.0,1,0,1,3,12.937220,80.60793,train
4,4,0,0,1,1.0,1,1,1,3,17.749338,86.02312,train


In [557]:
train_data = data[data['Dataset'] == 'train']
original_data = data[data['Dataset'] == 'original']
extra_data = data[data['Dataset'] == 'train_extra']
test_data = data[data['Dataset'] == 'test']
test_data = test_data.drop(target_column, axis=1)

test_id = test_data[['id']].copy()
train_data = train_data.drop('id', axis=1)
original_data = original_data.drop('id', axis=1)
extra_data = extra_data.drop('id', axis=1)
test_data = test_data.drop('id', axis=1)

In [558]:
def getKFoldScores(kf, train_data, params):
    X = train_data.drop(target_column, axis=1)
    y = (train_data[target_column])

    scores = []
    for i, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        cat_columns = ['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Size', 'Style', 'Color', 'Dataset']
        train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_columns)
        val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=cat_columns)
        
        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
        )

        scores.append(model.best_score['valid_0']['rmse'])
    return scores

In [559]:
kf = KFold(n_splits=5)

In [560]:
params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'n_jobs': -1
}

train_scores = getKFoldScores(kf, train_data, params)
original_scores = getKFoldScores(kf, original_data, params)
extra_scores = getKFoldScores(kf, extra_data, params)

print("Mean RMSE (train):", np.mean(train_scores))
print("Mean RMSE (original):", np.mean(original_scores))
print("Mean RMSE (extra):", np.mean(extra_scores))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001725 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 305
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 81.474972
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001880 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 305
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 9
[L

In [561]:
def getPrediction(estimator, test_id, test_data, target):
    y_pred_submission = estimator.predict(test_data)
    
    test_submission = test_id
    test_submission[target] = y_pred_submission

    return test_submission

In [562]:
def createSubmission(data, params, type_str):
    X = data.drop(target_column, axis=1)
    y = (data[target_column])
    data = lgb.Dataset(X, label=y)

    model = lgb.train(
            params,
            data
        )
    
    test_submission = getPrediction(model, test_id, test_data, target_column)
    test_submission.to_csv(f"lgbm_gbdt_{type_str}.csv", index=False)

In [563]:
createSubmission(train_data, params, "train")
createSubmission(original_data, params, "original")
createSubmission(extra_data, params, "extra")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002471 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 305
[LightGBM] [Info] Number of data points in the train set: 300000, number of used features: 9
[LightGBM] [Info] Start training from score 81.411107
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000517 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 52500, number of used features: 9
[LightGBM] [Info] Start training from score 77.820789
[LightGBM] [Warning] Found wh